In [91]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, MetaData, Table, and_, or_, func
import pandas as pd

In [159]:
# create an engine which bindes to our sqlite instance
engine = create_engine('sqlite:///yelp.db', echo=False)

#get the declare a metadata instance to get the metadata off  our tables
meta = MetaData(engine)

#represent our sql tables as python objects
business = Table('business', meta, autoload=True)
users = Table('users',meta,autoload=True)
reviews = Table('reviews',meta,autoload=True)

#establish session with our database
DBSession = sessionmaker(bind=engine)
session = DBSession()

# Finding states with the most registered businesses
- keeping states with 10K+ business establishments

In [114]:
query = ( 
    session.query(business.c.state,func.count(business.c.business_id).label('num_businesses'))
    .group_by(business.c.state)
    .having(func.count(business.c.business_id) > 10000)
    .order_by(func.count(business.c.business_id).desc()) 
    
    )

pd.read_sql(query.statement,session.bind)

,state,num_businesses
0,MA,36012
1,OR,25175
2,TX,24485
3,FL,21907
4,GA,18090
5,BC,17298
6,OH,11258


# Question 1: Which states have the most business listings, reviews and average stars?

In [166]:
query = ( 
    session.query(business.c.state,
                  func.count(business.c.business_id).label('num_businesses'),
                  func.avg(business.c.stars).label('avg_stars'),
                  func.avg(business.c.review_count).label('num_reviews'))
    .group_by(business.c.state)
    .having(func.count(business.c.business_id) > 10000)
    .order_by(func.avg(business.c.stars).desc(),
             func.avg(business.c.review_count).desc()) 
    )


pd.read_sql(query.statement,session.bind)

,state,num_businesses,avg_stars,num_reviews
0,OR,25175,3.863873,55.580457
1,TX,24485,3.846314,59.398285
2,MA,36012,3.618211,55.909058
3,FL,21907,3.554343,49.979276
4,OH,11258,3.533043,36.802363
5,BC,17298,3.512574,34.818650
6,GA,18090,3.507822,61.505583


# Question 2 and 3: Finding the most popular restaurant in Oregon

In [174]:
query = (
    session.query(business)
    .filter(business.c.state == 'OR')
    .filter(business.c.city == 'Portland')
    .order_by(business.c.review_count.desc())
    .limit(1)
)

pd.read_sql(query.statement,session.bind)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,business_attribute_id
0,4CxF8c3MB7VAdY8zFb2cZQ,Voodoo Doughnut - Old Town,22 SW 3rd Ave,Portland,OR,97204,45.522612,-122.673084,3.5,9185,1,4CxF8c3MB7VAdY8zFb2cZQ


### We can easily get the id of this restaurant and all of its corresponding attributes as shown:



In [179]:
top_business = (
    session.query(business)
    .filter(business.c.state == 'OR')
    .filter(business.c.city == 'Portland')
    .order_by(business.c.review_count.desc())
    .first()
)

top_business.name

'Voodoo Doughnut - Old Town'

In [180]:
top_business.review_count

9185

In [181]:
top_business.business_id

'4CxF8c3MB7VAdY8zFb2cZQ'

# Questions 4 and 5: Finding the most negative reviewer for this restaurant

In [192]:
query = (
    session.query(reviews,
                 func.count(reviews.c.user_id).label('number_of_reviews'))
    .filter(reviews.c.business_id == top_business.business_id)
    .group_by(reviews.c.user_id)
    .order_by(func.count(reviews.c.user_id).desc(),
              func.avg(reviews.c.stars).asc())
    .limit(3)
)
pd.read_sql(query.statement,session.bind)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,number_of_reviews
0,5nbY1JO2l27lZIN8gvIC7A,Q2u4PQ5_aMBAQtyX19C1Iw,4CxF8c3MB7VAdY8zFb2cZQ,3.0,0,0,1,Their donuts are interesting to look at but I ...,2019-02-27 03:29:35,3
1,Zsj3nsvclL2OgOmLRdmWfg,vsmhQJ5jkw9wQjfpcOfuJA,4CxF8c3MB7VAdY8zFb2cZQ,2.0,0,0,0,Sorry VooDoo. You have fallen prey to the tou...,2011-03-20 22:16:57,3
2,gpodEglDZd7u_A1-xqccXg,m2XKqIaqB2P5GGm7cqZr2A,4CxF8c3MB7VAdY8zFb2cZQ,5.0,5,5,6,No visit to Portland is complete without stopp...,2011-08-04 16:24:06,3


### finding the negative reviewer

In [230]:
#returns a sql alchemy row object that contains the result
neg_reviewer = (
    session.query(reviews,
                 func.count(reviews.c.user_id).label('number_of_reviews'))
    .filter(reviews.c.business_id == top_business.business_id)
    .group_by(reviews.c.user_id)    
    .order_by(func.count(reviews.c.user_id).desc(),
              func.avg(reviews.c.stars).asc())
    .first()
)

### every result set becomes a row object

In [229]:
type(neg_reviewer)

sqlalchemy.engine.row.Row

### we can access the attributes of the row like a Python object

In [227]:
neg_reviewer.user_id

'Q2u4PQ5_aMBAQtyX19C1Iw'

### we can now dynamically populate sqlalchemy queries with row object attributes (ie. neg_reviewer.user_id)

In [232]:
query = (
    session.query(users.c.name.label('user_name'),reviews)
    .join(users, reviews.c.user_id == users.c.user_id)
    .filter(users.c.user_id == neg_reviewer.user_id)
)

pd.read_sql(query.statement,session.bind).head(3)

,user_name,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,David,S02GSYVuVx7PqnEi8OLiUg,Q2u4PQ5_aMBAQtyX19C1Iw,H_RM2u1WWGU1HkKZrYq2Ow,1.0,0,0,0,I don't care for their donuts at all. I first ...,2019-02-27 03:32:57
1,David,G1O3L_hGb_LsVolG_hAUmg,Q2u4PQ5_aMBAQtyX19C1Iw,w1tTsQKoPg5qBE7swhkXlg,5.0,1,0,1,Svetlana is a great masseuse. I go for massage...,2019-06-22 22:26:03
2,David,iZ4AYmnSQescTIjS7uUB3w,Q2u4PQ5_aMBAQtyX19C1Iw,z1QhExIrJH0cnBdQKrsk0g,5.0,0,0,0,Its a very lovely restaurant with an excellent...,2019-06-27 06:28:02
